In [1]:
from cideMOD import (
    CellParser,
    ErrorCheck,
    NDProblem,
    Problem,
    SolverCrashed,
    Trigger,
    init_results_folder,
    ModelOptions,
    bms
)
import json 

In [2]:
overwrite = True
case = "Safari_2009"
data_path = "../data/data_{}".format(case)

model_options = ModelOptions(mode='P2D', clean_on_exit=False, solve_SEI=True)

save_path = init_results_folder(
    case, overwrite=overwrite, copy_files=[f"{data_path}/params_OCV.json"]
)

cell_ocv = CellParser("params_OCV.json", data_path=data_path)
cell_cy = CellParser("params_cycling.json", data_path=data_path)
cell = json.load(open(data_path+"/params_cycling.json"))

Saving results to /home/fenics/shared/cideMOD/examples/results_Safari_2009
Capacidad Anodo: 2.295594689447843
Capacidad Catodo: 1.9868852574347429
Capacidad Celda: 1.9868852574347429
Capacidad Anodo: 1.6977835724041341
Capacidad Catodo: 1.9868852574347429
Capacidad Celda: 1.6977835724041341


## Cycling

In [3]:
V_app = 3.9
I_app = 0

days = 484
hours = days * 24
minutes = hours * 60
seconds = minutes * 60
t_f = seconds

In [4]:
cycling = {
            "name": "Discharge Cycle",
            "type": "Cycle",
            "count": 50,
            "steps": [
                {
                    "name": "Discharge",
                    "type": "Current", 
                    "value": "-0.5*1.8", 
                    "unit": "A", 
                    "t_max": {"value": 2.5, "unit": "h"},
                    "store_delay": -1 ,
                    "min_step": 2,
                    "events": [
                        {
                            "type": "Voltage",
                            "value": 2,  
                            "unit": "V",  
                            "atol": 1e-4, 
                            "rtol": 1e-3,
                            "goto": "Next" 
                        }
                    ]
                },
                {
                    "name": "Charge",
                    "type": "Current", 
                    "value": "0.5*1.8", 
                    "unit": "A", 
                    "t_max": {"value": 2.5, "unit": "h"},
                    "store_delay": -1 ,
                    "min_step": 5,
                    "events": [
                        {
                            "type": "Voltage",
                            "value": 4.2,  
                            "unit": "V",  
                            "atol": 1e-4, 
                            "rtol": 1e-3,
                            "goto": "CV" 
                        }
                    ]
                }
            ]}

cycling_test_plan = {
    'initial_state': {
        'SOC': 1,
        'exterior_temperature': 298
    },
    'steps': [cycling]
}

### Reaction limited case

In [5]:
cell["negativeElectrode"]["SEI"]["solventDiffusion"]["value"].update(3.7e-19)
cell["negativeElectrode"]["SEI"]["rateConstant"]["value"].update(1.36e-12)
bms_rl = bms.BMS(cell, model_options, data_path, name=case, overwrite=overwrite)
bms_rl.read_test_plan(cycling_test_plan)
bms_rl.run_test_plan()

problem_rl = bms_rl.problem

KeyError: 'negative_electrode'

### Diffusion limited case

In [ ]:
cell["negativeElectrode"]["SEI"]["solventDiffusion"]["value"].update(6.8e-21)
cell["negativeElectrode"]["SEI"]["rateConstant"]["value"].update(1.36e-7)
bms_dl = bms.BMS(cell, model_options, data_path, name=case, overwrite=overwrite)
bms_dl.read_test_plan(cycling_test_plan)
bms_dl.run_test_plan()

problem_dl = bms_dl.problem

# Compare results

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy
from scipy.interpolate import interp1d
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

safari_rl = pd.read_csv("../data/data_Safari_2009/SEI_resistance_rl.txt", header=None, sep='\t').to_numpy()
safari_dl = pd.read_csv("../data/data_Safari_2009/SEI_resistance_dl.txt", header=None, sep='\t').to_numpy()

### Capacity loss

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4), dpi=200)

# Reaction limited
cur = problem_rl.WH.global_var_arrays[2]
index = numpy.append(numpy.where(numpy.diff(cur)<(-0.5*1.8))[0], len(cur)-1)
nCycles = len(index)
cycles_rl = numpy.arange(1,nCycles+1)
thickness = problem_rl.WH.global_var_arrays[-1]
av_thickness_rl = numpy.array(thickness)[index]

ax1.plot(safari_rl[:,0], safari_rl[:,1], "-")
ax1.plot(cycles_rl, av_thickness_rl/problem_rl.anode.SEI.kappa, "-")
ax1.set_xlabel("Cycles")
ax1.set_ylabel("SEI resistance [$\Omega m^2$]")
ax1.legend(["Safari", "cideMOD"], loc="best")
ax1.set_xlim([0, 50])
ax1.title.set_text("Kinetic-limited")

cur = problem_dl.WH.global_var_arrays[2]
index = numpy.append(numpy.where(numpy.diff(cur)<(-0.5*1.8))[0], len(cur)-1)
nCycles = len(index)
cycles_dl = numpy.arange(1,nCycles+1)
thickness = problem_dl.WH.global_var_arrays[-1]
av_thickness_dl = numpy.array(thickness)[index]

ax2.plot(safari_dl[:,0], safari_dl[:,1], "-")
ax2.plot(cycles_dl, av_thickness_dl/problem_dl.anode.SEI.kappa, "-")
ax2.set_xlabel("Cycles")
ax2.set_ylabel("SEI resistance [$\Omega m^2$]")
ax2.legend(["Safari", "cideMOD"], loc="best")
ax2.set_xlim([0, 50])
ax2.title.set_text("Diffusion-limited")

plt.tight_layout()
plt.show()

### Diffusion limited case

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4), dpi=200)
time = numpy.linspace(0, 50, 1000)

real_dl = interp1d(safari_dl[:,0], safari_dl[:,1], bounds_error=False)
cidemod_dl = interp1d(cycles_dl, av_thickness_dl/problem_dl.anode.SEI.kappa, bounds_error=False)
real_rl = interp1d(safari_rl[:,0], safari_rl[:,1], bounds_error=False)
cidemod_rl = interp1d(cycles_rl, av_thickness_rl/problem_rl.anode.SEI.kappa, bounds_error=False)

# plot the RMSE over cycles
ax2.plot(time, numpy.sqrt((real_rl(time)-cidemod_rl(time))**2/len(time)), "-")
ax2.set_xlabel("Cycles")
ax2.set_ylabel("RMSE [-]")

# plot the RMSE over cycles
ax2.plot(time, numpy.sqrt((real_dl(time)-cidemod_dl(time))**2/len(time)), "-")
ax2.set_xlabel("Cycles")
ax2.set_ylabel("RMSE [-]")

plt.tight_layout()
plt.show()

## Compare cycling parameters and OCV parameters

In [ ]:
safari_Q_cycling = pd.read_csv("../data/data_Safari_2009/capacity_loss_cy.txt", header=None, sep='\t').to_numpy()
safari_Q_OCV = pd.read_csv("../data/data_Safari_2009/capacity_loss.txt", header=None, sep='\t').to_numpy()

#### OCV

In [ ]:
problem_ocv = Problem(cell_ocv, model_options, save_path=save_path)
problem_ocv.set_cell_state(0.99, 273 + 25, 273 + 25)
problem_ocv.setup()

v_min = Trigger(2.5, "v")
status = problem_ocv.solve_ie(
    min_step=5, max_step=30*24*3600, i_app=I_app, t_f=t_f, store_delay=-1, adaptive=True, triggers=[v_min]
)
err = ErrorCheck(problem_ocv, status)

if isinstance(status, SolverCrashed):
    raise status.args[0]

#### Cycling

In [ ]:
problem_cy = Problem(cell_cy, model_options, save_path=save_path)
problem_cy.set_cell_state(0.99, 273 + 25, 273 + 25)
problem_cy.setup()

v_min = Trigger(2.5, "v")
status = problem_cy.solve_ie(
    min_step=5, max_step=30*24*3600, i_app=I_app, t_f=t_f, store_delay=-1, adaptive=True, triggers=[v_min]
)
err = ErrorCheck(problem_cy, status)

if isinstance(status, SolverCrashed):
    raise status.args[0]

#### Plot

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4), dpi=250)

# plot results over time
ax1.plot(safari_Q_cycling[:,0], safari_Q_cycling[:,1]*100, "-")
ax1.plot(numpy.array(problem_cy.WH.global_var_arrays[0])/3600/24, numpy.array(problem_cy.WH.global_var_arrays[-2])/problem_cy.cell.capacity*100, "-")
ax1.set_xlabel("Time [days]")
ax1.set_ylabel("Capacity loss [%]")
ax1.legend(["Safari", "cideMOD"], loc="best")
ax1.set_xlim([0, max(safari_Q_cycling[:,0])])

ax2.plot(safari_Q_OCV[:,0], safari_Q_OCV[:,1]*100, "-")
ax2.plot(numpy.array(problem_ocv.WH.global_var_arrays[0])/3600/24, numpy.array(problem_ocv.WH.global_var_arrays[-2])/problem_ocv.cell.capacity*100, "-")
ax2.set_xlabel("Time [days]")
ax2.set_ylabel("Capacity loss [%]")
ax2.legend(["Safari", "cideMOD"], loc="best")
ax2.set_xlim([0, max(safari_Q_OCV[:,0])])

plt.tight_layout()
plt.show()